# Librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
%matplotlib inline

# Funciones


In [ ]:
def num_to_string(zcopy):
  # Definir el mapeo de valores numéricos para cada categoría
  mapeo_valores = {
    1 : 'bajo',
    2 : 'medio-bajo',
    3 : 'medio-alto',
    4 : 'alto',
  }
  zcopy["RENDIMIENTO_GLOBAL"] = zcopy["RENDIMIENTO_GLOBAL"].replace(mapeo_valores)
  return zcopy



def string_to_num(zcopy):
  # Definir el mapeo de valores numéricos para cada categoría
  mapeo_valores = {
    'bajo': 1,
    'medio-bajo': 2,
    'medio-alto': 3,
    'alto': 4,
  }
  zcopy["RENDIMIENTO_GLOBAL"] = zcopy["RENDIMIENTO_GLOBAL"].replace(mapeo_valores)
  return zcopy



def mostrar_valores_unicos(dataframe, columna):
  valores_unicos = dataframe[columna].unique()
  print(f"Valores únicos en la columna {columna}:", valores_unicos.size)
  print(valores_unicos)



def reemplazar_si_no(dataframe, columna):
  # Definir el mapeo de valores numéricos para cada categoría
  mapeo_valores = {
    'Si': 1,
    'No': 0,
  }
  # Reemplazar los valores en la columna FAMI_ESTRATOVIVIENDA por los valores numéricos
  dataframe[columna] = dataframe[columna].replace(mapeo_valores)



def contar_nulos_si(dataframe, columna1, valor1, columna2):
  # Crear una máscara para identificar las filas donde ESTU_HORASSEMANATRABAJA es igual a cero
  mascara = (dataframe[columna1] == valor1)

  # Contar los valores nulos en la columna ESTU_PAGOMATRICULAPROPIO donde la máscara es True
  cantidad_nulos = dataframe.loc[mascara, columna2].isna().sum()
  print("Cantidad de valores nulos en ", columna2, " cuando ",columna1, " es ", valor1, ": ", cantidad_nulos)
  return None



def rellenar_nulos(dataframe, columna1, valor1, columna2, valor2):
  # Crear una máscara para identificar las filas donde ESTU_HORASSEMANATRABAJA es igual a cero
  mascara = (dataframe[columna1] == valor1)

  # Rellenar los valores nulos en la columna ESTU_PAGOMATRICULAPROPIO con cero en las filas donde la máscara es True
  dataframe.loc[mascara, columna2] = dataframe.loc[mascara, columna2].fillna(valor2)

  return None



def rellenar_nulos2(dataframe, columna1, valor1, columna2, valor2, columna3, valor3):

  # Crear una máscara para identificar las filas donde las dos columnas tienen los valores específicos
  mascara = (dataframe[columna1] == valor1) & (dataframe[columna2] == valor2)

  # Rellenar los valores nulos en la columna3 con valor3 en las filas donde la máscara es True
  dataframe.loc[mascara, columna3] = dataframe.loc[mascara, columna3].fillna(valor3)

  return None



def rellenar_nulos3(dataframe, columna1, valor1, columna2, valor2, columna3, valor3, columna4, valor4):

  # Crear una máscara para identificar las filas donde las tres columnas tienen los valores específicos
  mascara = (dataframe[columna1] == valor1) & (dataframe[columna2] == valor2) & (dataframe[columna3] == valor3)

  # Rellenar los valores nulos en la columna4 con valor4 en las filas donde la máscara es True
  dataframe.loc[mascara, columna4] = dataframe.loc[mascara, columna4].fillna(valor4)

  return None



def mostrar_valores_unicos_y_frecuencia(dataframe, columna, valores_a_mostrar=50):

  conteo_valores = dataframe[columna].value_counts(dropna=False)

  print("Conteo de valores en la columna ", columna, ": "  , conteo_valores.size)
  print(conteo_valores)
  print()
  print()
  conteo_valores.head(valores_a_mostrar)

  return None

#TOMADAS DEL NOTEBOOK ELABORADO POR EL PROFESOR RAUL
def to_onehot(x):
  values = np.unique(x)
  r = np.r_[[np.argwhere(i==values)[0][0] for i in x]]
  return np.eye(len(values))[r].astype(int)

def replace_columns_with_onehot(d, col):
  k = to_onehot(d[col].values)
  r = pd.DataFrame(k, columns=["%s_%d"%(col, i) for i in range(k.shape[1])], index=d.index).join(d)
  del(r[col])
  return r

# Graficas

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def grafico_barras(df, ejex, ejey, inicio=0, elementos=0, tamaño_x=12, tamaño_y=6):

  # Crear un gráfico de barras
  #plt.figure(figsize=(tamaño_x, tamaño_y))
  num_categorias = len(df[ejex].value_counts())
  tamaño_x = min(tamaño_x, max(tamaño_y, num_categorias * 0.75))  # Tamaño mínimo de 6, máximo de 12
  tamaño_y = tamaño_x * 0.75

  # Crear un gráfico de barras
  plt.figure(figsize=(tamaño_x, tamaño_y))

  if elementos == 0:
    sns.countplot(data=df, x=ejex, hue=ejey, palette="Set2", order=df[ejex].value_counts().index)
  else:
    sns.countplot(data=df, x=ejex, hue=ejey, palette="Set2", order=df[ejex].value_counts().index[inicio:elementos])

  # Añadir etiquetas y título
  plt.xlabel(ejex)
  plt.ylabel(ejey)
  plt.title("Distribución del " + ejey + " por " + ejex)

  # Rotar las etiquetas del eje x para una mejor visualización
  plt.xticks(rotation=90)

  # Ajustar la leyenda
  plt.legend(title=ejey, loc="upper right")

  # Mostrar el gráfico
  plt.tight_layout()
  plt.show()

  return None

#Carga de datos

In [ ]:
#recordar ingresar el token.json
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242
!unzip udea*.zip > /dev/null

 45% 9.00M/20.1M [00:00<00:00, 56.0MB/s]
100% 20.1M/20.1M [00:00<00:00, 98.6MB/s]


In [ ]:
z = pd.read_csv('train.csv')
dts = pd.read_csv("test.csv")
dtr = z.copy()

dtr.shape, dts.shape

((692500, 12), (296786, 12))

#Limpieza (Preprocesado)

In [ ]:
import pandas as pd
zcopy = z.copy()


In [ ]:
# Inspección de los Departamentos que hay como respuesta en el Dataset
departamentos_unicos = zcopy['ESTU_PRGM_DEPARTAMENTO'].unique()
print(departamentos_unicos)


['BOGOTÁ' 'ATLANTICO' 'SANTANDER' 'ANTIOQUIA' 'HUILA' 'SUCRE' 'CAQUETA'
 'CUNDINAMARCA' 'BOLIVAR' 'TOLIMA' 'VALLE' 'QUINDIO' 'RISARALDA' 'CORDOBA'
 'META' 'LA GUAJIRA' 'BOYACA' 'NARIÑO' 'CAUCA' 'NORTE SANTANDER' 'CESAR'
 'PUTUMAYO' 'CALDAS' 'MAGDALENA' 'CHOCO' 'CASANARE' 'ARAUCA' 'GUAVIARE'
 'AMAZONAS' 'VAUPES' 'SAN ANDRES']


In [ ]:
# Inspección de las carreras que hay como respuesta en el Dataset
lista_carreras = zcopy['ESTU_PRGM_ACADEMICO'].unique()
print(lista_carreras)



['ENFERMERIA' 'DERECHO' 'MERCADEO Y PUBLICIDAD'
 'ADMINISTRACION DE EMPRESAS' 'PSICOLOGIA' 'MEDICINA VETERINARIA'
 'INGENIERIA MECANICA' 'ADMINISTRACIÓN EN SALUD OCUPACIONAL'
 'INGENIERIA INDUSTRIAL' 'ADMINISTRACIÓN FINANCIERA' 'HOTELERIA Y TURISMO'
 'LICENCIATURA EN CIENCIAS SOCIALES' 'LICENCIATURA EN PEDAGOGIA INFANTIL'
 'COMUNICACION SOCIAL' 'CIENCIA POLITICA'
 'PROFESIONAL EN GESTIÓN DE LA SEGURIDAD Y LA SALUD LABORAL'
 'MAESTRO EN MÚSICA' 'INGENIERIA MECATRONICA' 'TRABAJO SOCIAL'
 'LICENCIATURA EN BIOLOGIA Y EDUCACION AMBIENTAL' 'INGENIERIA CIVIL'
 'CONTADURIA PÚBLICA' 'ADMINISTRACION EN SALUD'
 'ADMINISTRACIÓN DE EMPRESAS' 'ESTADISTICA' 'LICENCIATURA EN BIOLOGIA'
 'INGENIERIA AGROINDUSTRIAL' 'ZOOTECNIA' 'COMUNICACION AUDIOVISUAL'
 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES-INGLES'
 'COMUNICACION SOCIAL  - PERIODISMO' 'SEGURIDAD Y SALUD EN EL TRABAJO'
 'MEDICINA' 'ADMINISTRACION DE LA SEGURIDAD SOCIAL' 'CONTADURIA PUBLICA'
 'ODONTOLOGIA' 'DISEÑO INDUSTRIAL' 'ECON

In [ ]:
# Inspección de los valores de matrícula que hay como respuesta en el Dataset
lista_pago_matricula = zcopy['ESTU_VALORMATRICULAUNIVERSIDAD'].unique()
print(lista_pago_matricula)

['Entre 5.5 millones y menos de 7 millones'
 'Entre 2.5 millones y menos de 4 millones'
 'Entre 4 millones y menos de 5.5 millones' 'Más de 7 millones'
 'Entre 1 millón y menos de 2.5 millones'
 'Entre 500 mil y menos de 1 millón' 'Menos de 500 mil'
 'No pagó matrícula' nan]


In [ ]:
# Inspección de los valores de educación del padre y madre que hay como respuesta en el Dataset
educacion_padre = zcopy['FAMI_EDUCACIONPADRE'].unique()
print(educacion_padre)



['Técnica o tecnológica incompleta' 'Técnica o tecnológica completa'
 'Secundaria (Bachillerato) completa' 'No sabe' 'Primaria completa'
 'Educación profesional completa' 'Educación profesional incompleta'
 'Primaria incompleta' 'Postgrado' nan
 'Secundaria (Bachillerato) incompleta' 'Ninguno' 'No Aplica']


In [ ]:
# Inspección de los valores de horas de trabajo que hay como respuesta en el Dataset
horas_trabajo = zcopy['ESTU_HORASSEMANATRABAJA'].unique()
print(horas_trabajo)

['Menos de 10 horas' '0' 'Más de 30 horas' 'Entre 21 y 30 horas'
 'Entre 11 y 20 horas' nan]


In [ ]:
def limpieza(dframe, train=True):
  zcopy = dframe.copy()
#----------------------------------BORRAR COLUMNAS----------------------------------

  #BORRAR COLUMNA ID: Identificador único de cada entrada. No aporta información predictiva.
  zcopy.drop(columns=["ID"], inplace=True)
  #BORRAR COLUMNA PERIODO: No aporta información predictiva.
  zcopy.drop(columns=["PERIODO"], inplace=True)

#----------------------------------MAPEAR COLUMNAS----------------------------------

  # Definir el mapeo de valores numéricos para ESTU_VALORMATRICULAUNIVERSIDAD
  mapeo_valores = {
    'Entre 5.5 millones y menos de 7 millones':6,
    'Entre 2.5 millones y menos de 4 millones':4,
    'Entre 4 millones y menos de 5.5 millones':5,
    'Más de 7 millones':7,
    'Entre 1 millón y menos de 2.5 millones':3,
    'Entre 500 mil y menos de 1 millón':2,
    'Menos de 500 mil':1,
    'No pagó matrícula':0,

  }
  zcopy["ESTU_VALORMATRICULAUNIVERSIDAD"] = zcopy["ESTU_VALORMATRICULAUNIVERSIDAD"].replace(mapeo_valores)

  # Definir el mapeo de valores numéricos para FAMI_EDUCACIONPADRE y FAMI_EDUCACIONMADRE
  mapeo_valores = {
    'No sabe' :0,
    'Ninguno':0,
    'No Aplica':0,
    'Primaria incompleta':0,
    'Primaria completa':1,
    'Secundaria (Bachillerato) incompleta' :1,
    'Secundaria (Bachillerato) completa':2,
    'Técnica o tecnológica incompleta':2,
    'Educación profesional incompleta':2,
    'Técnica o tecnológica completa':3,
    'Educación profesional completa' :4,
    'Postgrado':5,

  }

  zcopy["FAMI_EDUCACIONPADRE"] = zcopy["FAMI_EDUCACIONPADRE"].replace(mapeo_valores)
  zcopy["FAMI_EDUCACIONMADRE"] = zcopy["FAMI_EDUCACIONMADRE"].replace(mapeo_valores)

  # Definir el mapeo de valores numéricos para ESTU_HORASSEMANATRABAJA
  mapeo_valores = {
    '0': 0,
    'Menos de 10 horas': 1,
    'Entre 11 y 20 horas': 2,
    'Entre 21 y 30 horas': 3,
    'Más de 30 horas': 4,

  }

  zcopy["ESTU_HORASSEMANATRABAJA"] = zcopy["ESTU_HORASSEMANATRABAJA"].replace(mapeo_valores)

  # Definir el mapeo de valores numéricos para FAMI_ESTRATOVIVIENDA
  mapeo_valores = {
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6,
    'Sin Estrato': 0,
  }
  zcopy["FAMI_ESTRATOVIVIENDA"] = zcopy["FAMI_ESTRATOVIVIENDA"].replace(mapeo_valores)

  # Definir el mapeo de valores numéricos para cada categoría
  columnas_si_no = ["FAMI_TIENEINTERNET","ESTU_PAGOMATRICULAPROPIO"]
  for columna in columnas_si_no:
    reemplazar_si_no(zcopy, columna)

#----------------------------------RELLENAR OBVIEDADES----------------------------------
  estratos = [4,5,6]
  for i in estratos:
    rellenar_nulos(zcopy, "FAMI_ESTRATOVIVIENDA", i, "FAMI_TIENEINTERNET", 1)

  rellenar_nulos(zcopy, "ESTU_HORASSEMANATRABAJA", 0, "ESTU_PAGOMATRICULAPROPIO", 0)

#----------------------------------ELIMINAR FILAS CON VALORES NAN----------------------------------
  if train:
    umbral = 8
    zcopy = zcopy[zcopy.isnull().sum(axis=1) < umbral]

  zcopy = zcopy.fillna(0)

#----------------------------------AGRUPAR CARRERAS POR FACULTADES----------------------------------
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('INGENIERIA|INGENIER¿A|INGENIERÍA|acui|ing|compu', case=False), 'ESTU_PRGM_ACADEMICO'] = 'ingenieria'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('licenciatura|peda', case=False), 'ESTU_PRGM_ACADEMICO'] = 'educacion'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('econom|contadur|empresas|contabilidad|internaci|finan|comerci|negocios|mark|empr|merc', case=False), 'ESTU_PRGM_ACADEMICO'] = 'economia y negocios'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('derecho|política|politica|crimin|investi|gob|pol|juris|desa', case=False), 'ESTU_PRGM_ACADEMICO'] = 'derecho'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('social|psicolog|sicolog|psi|periodismo|antropolog|fami|sociol|geog', case=False), 'ESTU_PRGM_ACADEMICO'] = 'ciencias sociales'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('veteri|zoo|agro', case=False), 'ESTU_PRGM_ACADEMICO'] = 'veterinaria'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('medic|salud|enfer|odonto|fisio|nutri|quiru|terap|fonoaud|bacte|geron|optom', case=False), 'ESTU_PRGM_ACADEMICO'] = 'salud'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('dise|art|mus|audi|teat|publicidad|produ|cine|dram|danz|maestro|foto|publ|mús|anim|band|mueb', case=False), 'ESTU_PRGM_ACADEMICO'] = 'artes'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('admi|hote|turi', case=False), 'ESTU_PRGM_ACADEMICO'] = 'administracion'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('gastro|culin', case=False), 'ESTU_PRGM_ACADEMICO'] = 'gastronomia'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('farm', case=False), 'ESTU_PRGM_ACADEMICO'] = 'farmacia'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('arqui|const|urba', case=False), 'ESTU_PRGM_ACADEMICO'] = 'arquitectura'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('depor|recre', case=False), 'ESTU_PRGM_ACADEMICO'] = 'deporte'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('filo|hist|lite|ling|arque|cult|bibli|arch|leng|teo', case=False), 'ESTU_PRGM_ACADEMICO'] = 'humanidades'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('quim|fis|bio|geol|mat|micr|eco|ASTRONOMIA|quí|fí|estad|geo|ambi', case=False), 'ESTU_PRGM_ACADEMICO'] = 'exactas y naturales'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('milit|mari|nav', case=False), 'ESTU_PRGM_ACADEMICO'] = 'fuerzas armadas'
  zcopy.loc[zcopy['ESTU_PRGM_ACADEMICO'].str.contains('comu|narra|sis|info', case=False), 'ESTU_PRGM_ACADEMICO'] = 'comunicacion'

#----------------------------------AGRUPAR DEPARTAMENTOS POR REGIONES----------------------------------

  zcopy.loc[zcopy['ESTU_PRGM_DEPARTAMENTO'].str.contains('BOGOTÁ|SANTANDER|NORTE SANTANDER|ANTIOQUIA|HUILA|CUNDINAMARCA|TOLIMA|QUINDÍO|RISARALDA|BOYACÁ|CALDAS|', case=False), 'ESTU_PRGM_DEPARTAMENTO'] = 'andina'
  zcopy.loc[zcopy['ESTU_PRGM_DEPARTAMENTO'].str.contains('AMAZONAS|CAQUETÁ|GUAVIARE|PUTUMAYO|VAUPÉS', case=False), 'ESTU_PRGM_DEPARTAMENTO'] = 'amazonica'
  zcopy.loc[zcopy['ESTU_PRGM_DEPARTAMENTO'].str.contains('VALLE|CHOCO|NARIÑO|CAUCA', case=False), 'ESTU_PRGM_DEPARTAMENTO'] = 'pacifico'
  zcopy.loc[zcopy['ESTU_PRGM_DEPARTAMENTO'].str.contains('ARAUCA|CASANARE|META', case=False), 'ESTU_PRGM_DEPARTAMENTO'] = 'orinoquia'
  zcopy.loc[zcopy['ESTU_PRGM_DEPARTAMENTO'].str.contains('ATLÁNTICO|BOLÍVAR|CESAR|CÓRDOBA|LA GUAJIRA|MAGDALENA|SUCRE', case=False), 'ESTU_PRGM_DEPARTAMENTO'] = 'caribe'
  zcopy.loc[zcopy['ESTU_PRGM_DEPARTAMENTO'].str.contains('SAN ANDRÉS', case=False), 'ESTU_PRGM_DEPARTAMENTO'] = 'insular'

#----------------------------------LIMPIEZA ONE HOT----------------------------------
  zcopy = replace_columns_with_onehot(zcopy, "ESTU_PRGM_ACADEMICO")
  zcopy = replace_columns_with_onehot(zcopy, "ESTU_PRGM_DEPARTAMENTO")

  return zcopy

# Antes y despues de la limpieza

In [ ]:
zcopy = limpieza(dtr)

<ipython-input-14-c92b5525c82d>:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  zcopy["ESTU_VALORMATRICULAUNIVERSIDAD"] = zcopy["ESTU_VALORMATRICULAUNIVERSIDAD"].replace(mapeo_valores)
<ipython-input-14-c92b5525c82d>:43: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  zcopy["FAMI_EDUCACIONPADRE"] = zcopy["FAMI_EDUCACIONPADRE"].replace(mapeo_valores)
<ipython-input-14-c92b5525c82d>:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call 

In [ ]:
Xts = limpieza(dts, train= False)

<ipython-input-14-c92b5525c82d>:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  zcopy["ESTU_VALORMATRICULAUNIVERSIDAD"] = zcopy["ESTU_VALORMATRICULAUNIVERSIDAD"].replace(mapeo_valores)
<ipython-input-14-c92b5525c82d>:43: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  zcopy["FAMI_EDUCACIONPADRE"] = zcopy["FAMI_EDUCACIONPADRE"].replace(mapeo_valores)
<ipython-input-14-c92b5525c82d>:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call 

In [ ]:
zcopy = string_to_num(zcopy)
zcopy.head()

<ipython-input-3-8271cbeee73f>:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  zcopy["RENDIMIENTO_GLOBAL"] = zcopy["RENDIMIENTO_GLOBAL"].replace(mapeo_valores)


,ESTU_PRGM_DEPARTAMENTO_0,ESTU_PRGM_ACADEMICO_0,ESTU_PRGM_ACADEMICO_1,ESTU_PRGM_ACADEMICO_2,ESTU_PRGM_ACADEMICO_3,ESTU_PRGM_ACADEMICO_4,ESTU_PRGM_ACADEMICO_5,ESTU_PRGM_ACADEMICO_6,ESTU_PRGM_ACADEMICO_7,ESTU_PRGM_ACADEMICO_8,...,ESTU_PRGM_ACADEMICO_13,ESTU_PRGM_ACADEMICO_14,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,1,0,0,0,0,0,0,0,0,0,...,1,0,6.0,1.0,3.0,1.0,2.0,5.0,0.0,3
1,1,0,0,0,0,0,0,1,0,0,...,0,0,4.0,0.0,3.0,0.0,3.0,2.0,0.0,1
2,1,0,0,0,0,0,0,0,0,1,...,0,0,4.0,4.0,3.0,1.0,2.0,2.0,0.0,1
3,1,0,0,0,0,0,0,0,0,1,...,0,0,5.0,0.0,4.0,1.0,0.0,2.0,0.0,4
4,1,0,0,0,1,0,0,0,0,0,...,0,0,4.0,3.0,3.0,1.0,1.0,1.0,0.0,2


In [ ]:
print("Antes de la limpieza:")
print("dtrain", dtr.shape, "dtest", dts.shape)
print("Después de la limpieza:")
print("dtrain", zcopy.shape, "dtest", Xts.shape)

Antes de la limpieza:
dtrain (692500, 12) dtest (296786, 12)
Después de la limpieza:
dtrain (692500, 24) dtest (296786, 24)


# Split de los datos (train -> train/validation)

In [ ]:
submission = dts["ID"].values

ytr = zcopy["RENDIMIENTO_GLOBAL"].values
zcopy.drop(columns=["RENDIMIENTO_GLOBAL"], inplace=True)
Xtr = zcopy

In [ ]:
print("Xtrain       ytrain")
print (Xtr.shape, ytr.shape)
print("Xtest        submission")
print (Xts.shape, submission.shape)

Xtrain       ytrain
(692500, 23) (692500,)
Xtest        submission
(296786, 24) (296786,)


In [ ]:
# TRAIN 70% ---- TEST 30%
X_train, X_test, y_train, y_test = train_test_split(Xtr, ytr, test_size=0.3, random_state=42)
print("X_train       X_test       y_train   y_test")
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

X_train       X_test       y_train   y_test
(484750, 23) (207750, 23) (484750,) (207750,)


In [ ]:
print(X_train.dtypes)
print(X_train.iloc[:, 22].unique())


ESTU_PRGM_DEPARTAMENTO_0            int64
ESTU_PRGM_ACADEMICO_0               int64
ESTU_PRGM_ACADEMICO_1               int64
ESTU_PRGM_ACADEMICO_2               int64
ESTU_PRGM_ACADEMICO_3               int64
ESTU_PRGM_ACADEMICO_4               int64
ESTU_PRGM_ACADEMICO_5               int64
ESTU_PRGM_ACADEMICO_6               int64
ESTU_PRGM_ACADEMICO_7               int64
ESTU_PRGM_ACADEMICO_8               int64
ESTU_PRGM_ACADEMICO_9               int64
ESTU_PRGM_ACADEMICO_10              int64
ESTU_PRGM_ACADEMICO_11              int64
ESTU_PRGM_ACADEMICO_12              int64
ESTU_PRGM_ACADEMICO_13              int64
ESTU_PRGM_ACADEMICO_14              int64
ESTU_VALORMATRICULAUNIVERSIDAD    float64
ESTU_HORASSEMANATRABAJA           float64
FAMI_ESTRATOVIVIENDA              float64
FAMI_TIENEINTERNET                float64
FAMI_EDUCACIONPADRE               float64
FAMI_EDUCACIONMADRE               float64
ESTU_PAGOMATRICULAPROPIO          float64
dtype: object
[0. 1.]


# Modelo

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA


Catboost

In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.7 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

cb = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=10, random_seed=42, verbose=100)
cb.fit(X_train, y_train)

# Calcular y mostrar el puntaje en los datos de entrenamiento
train_score_cb = cb.score(X_train, y_train)
print("Score en datos de TRAIN:", train_score_cb)

preds_cb = cb.predict(X_test)

# Calcular y mostrar el puntaje en los datos de prueba
test_score_cb = accuracy_score(y_test, preds_cb)
print("Score en datos de TEST:", test_score_cb)

0:	learn: 1.3646562	total: 1.55s	remaining: 25m 54s
100:	learn: 1.2371683	total: 1m 18s	remaining: 11m 42s
200:	learn: 1.2257407	total: 2m 17s	remaining: 9m 4s
300:	learn: 1.2167207	total: 3m 11s	remaining: 7m 25s
400:	learn: 1.2090997	total: 4m 5s	remaining: 6m 6s
500:	learn: 1.2024266	total: 4m 59s	remaining: 4m 58s
600:	learn: 1.1964311	total: 5m 54s	remaining: 3m 55s
700:	learn: 1.1911229	total: 7m 1s	remaining: 2m 59s
800:	learn: 1.1862520	total: 8m 1s	remaining: 1m 59s
900:	learn: 1.1818858	total: 8m 57s	remaining: 59s
999:	learn: 1.1777549	total: 9m 50s	remaining: 0us
Score en datos de TRAIN: 0.4616709644146467
Score en datos de TEST: 0.3952202166064982


In [ ]:
from catboost import CatBoostClassifier

cb1 = CatBoostClassifier(iterations=800, learning_rate=0.05, depth=6, random_seed=42, verbose=100)
cb1.fit(X_train, y_train)

# Calcular y mostrar el puntaje en los datos de entrenamiento
train_score_cb1 = cb1.score(X_train, y_train)
print("Score en datos de TRAIN:", train_score_cb1)

preds_cb1 = cb1.predict(X_test)

# Calcular y mostrar el puntaje en los datos de prueba
test_score_cb1 = accuracy_score(y_test, preds_cb1)
print("Score en datos de TEST:", test_score_cb1)

0:	learn: 1.3773183	total: 520ms	remaining: 6m 55s
100:	learn: 1.2561015	total: 25.3s	remaining: 2m 55s
200:	learn: 1.2498887	total: 50.8s	remaining: 2m 31s
300:	learn: 1.2472087	total: 1m 16s	remaining: 2m 6s
400:	learn: 1.2452931	total: 1m 39s	remaining: 1m 39s
500:	learn: 1.2438391	total: 2m 5s	remaining: 1m 15s
600:	learn: 1.2426100	total: 2m 31s	remaining: 50.3s
700:	learn: 1.2415296	total: 2m 55s	remaining: 24.8s
799:	learn: 1.2405865	total: 3m 21s	remaining: 0us
Score en datos de TRAIN: 0.41180608561113974
Score en datos de TEST: 0.4057809867629362


In [ ]:
from catboost import CatBoostClassifier

cb2 = CatBoostClassifier(iterations=400, learning_rate=0.1, depth=6, random_seed=42, verbose=100)
cb2.fit(Xtr, ytr)

# Calcular y mostrar el puntaje en los datos de entrenamiento
train_score_cb2 = cb2.score(Xtr, ytr)
print("Score en datos de TRAIN:", train_score_cb2)

preds_cb2 = cb2.predict(X_test)

# Calcular y mostrar el puntaje en los datos de prueba
test_score_cb2 = accuracy_score(y_test, preds_cb2)
print("Score en datos de TEST:", test_score_cb2)

0:	learn: 1.3678557	total: 712ms	remaining: 4m 43s
100:	learn: 1.2497665	total: 36.6s	remaining: 1m 48s
200:	learn: 1.2456485	total: 1m 13s	remaining: 1m 12s
300:	learn: 1.2434657	total: 1m 47s	remaining: 35.5s
399:	learn: 1.2418497	total: 2m 23s	remaining: 0us
Score en datos de TRAIN: 0.41055451263537907
Score en datos de TEST: 0.41176895306859207



fill con el mejor modelo

In [ ]:
preds = cb2.predict(Xts)
preds = preds.ravel()

# Kaggle submission

In [ ]:
submission = pd.DataFrame([dts["ID"], pd.Series(preds, name="RENDIMIENTO_GLOBAL")]).T

submission = num_to_string(submission)
submission['ID'] = submission['ID'].astype(int)

submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,alto
1,98545,medio-bajo
2,499179,alto
3,782980,bajo
4,785185,medio-bajo


In [ ]:
submission.to_csv("my_submission.csv", index=False)

In [ ]:
!head my_submission.csv

ID,RENDIMIENTO_GLOBAL
550236,alto
98545,medio-bajo
499179,alto
782980,bajo
785185,medio-bajo
58495,bajo
705444,alto
557548,alto
519909,bajo


In [ ]:
submission.shape

(296786, 2)

In [ ]:
!kaggle competitions submit -c udea-ai4eng-20242 -f my_submission.csv -m "try 3"

100% 4.01M/4.01M [00:00<00:00, 8.60MB/s]
Successfully submitted to UDEA/ai4eng 20242 - Pruebas Saber Pro Colombia